In [1]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from scipy import stats
import math

In [2]:
initial = pd.read_csv('C:/users/rusty/documents/github/patchseq_gdoc_analysis/patch_seq_log_mouse.csv')

In [3]:
ps = initial[['Date', 'Morph', 'Post patch?', 'Post patch pipette R', 'SM_QC_PF', 'Bad dates']]
ps['Date'] = pd.to_datetime(ps['Date'], format = '%y%m%d')
ps['Morph'] = ps['Morph'].astype('category')
#ps['Bad dates'] = ps['Bad dates'].astype('category')
#ps['PCR cycles'] = ps['PCR cycles'].astype('category')
ps['Post patch?'] = ps['Post patch?'].astype('category')
ps['SM_QC_PF'] = ps['SM_QC_PF'].astype('category')
ps['Bad dates'] = ps['Bad dates'].astype('category')
#ps = ps[ps['PCR cycles'] == 21]
ps = ps[ps['Bad dates'] != 'x']
ps = ps[ps['SM_QC_PF'] != 'cnt']
ps = ps[(ps['Morph'] == 'y') | (ps['Morph'] == 'n') | (ps['Morph'] == 'q')]
ps.dropna(subset=['Morph', 'Post patch?'], inplace=True)
ps

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-do

,Date,Morph,Post patch?,Post patch pipette R,SM_QC_PF,Bad dates
70,2017-01-09,y,Outside-Out,NaN,pass,NaN
71,2017-01-09,n,Outside-Out,NaN,fail,NaN
77,2017-01-10,n,Outside-Out,NaN,pass,NaN
78,2017-01-10,y,Outside-Out,NaN,pass,NaN
79,2017-01-10,n,No-Seal,NaN,pass,NaN
80,2017-01-10,n,Outside-Out,NaN,fail,NaN
81,2017-01-10,q,Outside-Out,NaN,fail,NaN
82,2017-01-10,n,Outside-Out,NaN,fail,NaN
89,2017-01-11,n,No-Seal,NaN,fail,NaN
90,2017-01-11,n,Partial-Nucleus,NaN,fail,NaN


In [4]:
ps = ps.drop('Bad dates', 1)
ps = ps.drop('SM_QC_PF', 1)
ps.head()

,Date,Morph,Post patch?,Post patch pipette R
70,2017-01-09,y,Outside-Out,NaN
71,2017-01-09,n,Outside-Out,NaN
77,2017-01-10,n,Outside-Out,NaN
78,2017-01-10,y,Outside-Out,NaN
79,2017-01-10,n,No-Seal,NaN


In [5]:
#ps['Post patch?'].value_counts()

In [6]:
def nucleated(x):
    nucleus = x[(x["Post patch?"] == "Nucleated") | (((x["Post patch?"] == "nucleus_visible") | (x["Post patch?"] == "nucleus_present")) & (x["Post patch pipette R"] >= 500))]
    return nucleus

def partial_nucleated(y):
    partial = y[(y["Post patch?"] == 'Partial-Nucleus') | (((y['Post patch?'] == 'nucleus_present') | (y['Post patch?'] == 'nucleus_visible')) & (y["Post patch pipette R"] <= 499))]
    return partial

def outside_out(z):
    outside = z[(z["Post patch?"] == 'Outside-Out') | (((z['Post patch?'] == 'nucleus_absent') | (z['Post patch?'] == 'no_nuclues_visible')) & (z['Post patch pipette R'] >= 500))]
    return outside

def no_seal(w): 
    no = w[(w["Post patch?"] == 'No-Seal') | (((w['Post patch?'] == 'nucleus_absent') | (w['Post patch?'] == 'no_nucleus_visible')) & (w['Post patch pipette R'] <= 499))]
    return no
    
def entire_cell(v):
    entire = v[(v["Post patch?"] == 'Entire-Cell') | (v['Post patch?'] == 'entire_cell')]
    return entire

In [7]:
nuc = nucleated(ps)
nuc["post patch class"] = "a.Nucleated"
part_nuc = partial_nucleated(ps)
part_nuc["post patch class"] = "b.Partial nucleated"
out_out = outside_out(ps)
out_out["post patch class"] = "c.Outside out"
noSeal = no_seal(ps)
noSeal["post patch class"] = "d.No seal"
entirecell = entire_cell(ps)
entirecell['post patch class'] = "e.Entire cell"

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#inde

In [8]:
frames = [nuc, part_nuc, out_out, noSeal, entirecell]
ps = pd.concat(frames)
ps.head()

,Date,Morph,Post patch?,Post patch pipette R,post patch class
107,2017-01-12,q,Nucleated,NaN,a.Nucleated
110,2017-01-12,q,Nucleated,NaN,a.Nucleated
116,2017-01-12,q,Nucleated,NaN,a.Nucleated
118,2017-01-12,n,Nucleated,NaN,a.Nucleated
185,2017-01-17,y,Nucleated,NaN,a.Nucleated


In [9]:
ps = ps.drop('Post patch?', 1)
ps = ps.drop('Post patch pipette R', 1)
ps = ps.drop('Date', 1)
ps.head()

,Morph,post patch class
107,q,a.Nucleated
110,q,a.Nucleated
116,q,a.Nucleated
118,n,a.Nucleated
185,y,a.Nucleated


In [10]:
y = ps[ps['Morph'] == 'y']
y["morpho_call"] = "y"
n = ps[ps['Morph'] == 'n']
n['morpho_call'] = 'n'
q = ps[ps['Morph'] == 'q']
q['morpho_call'] = 'q'
frames = [y, n, q]
morphpp = pd.concat(frames)
#morphpp = morphpp.drop('Date', 1)
morphpp = morphpp.groupby(['morpho_call', 'post patch class']).count()
morphpp = morphpp.reset_index()
morphpp = morphpp.pivot_table(index = ['post patch class'], columns = 'morpho_call', 
                              values = ['Morph'], aggfunc = 'sum', fill_value=0)
morphpp
#ps['Morph'].value_counts()

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#inde

Morph          
morpho_call             n    q    y
post patch class                   
a.Nucleated           350  218  327
b.Partial nucleated   192   39   57
c.Outside out         175   63  179
d.No seal             619   35   67
e.Entire cell          17    3    4

In [11]:
morph_pcts = morphpp.div(morphpp.sum(1), axis = 0)*100
morph_pcts

Morph                      
morpho_call                  n          q          y
post patch class                                    
a.Nucleated          39.106145  24.357542  36.536313
b.Partial nucleated  66.666667  13.541667  19.791667
c.Outside out        41.966427  15.107914  42.925659
d.No seal            85.852982   4.854369   9.292649
e.Entire cell        70.833333  12.500000  16.666667

In [12]:
#nucs = list(morph_pcts['Morph', 'Nucleated'])
#part_nucs = list(morph_pcts['Morph', 'Partial nucleated'])
#out_outs = list(morph_pcts['Morph', 'Outside out'])
#no_seal = list(morph_pcts['Morph', 'No seal'])
#entire = list(morph_pcts['Morph', 'Entire cell'])
y = list(morph_pcts['Morph', 'y'])
q = list(morph_pcts['Morph', 'q'])
n = list(morph_pcts['Morph', 'n'])
y

[36.536312849162009,
 19.791666666666664,
 42.925659472422062,
 9.2926490984743406,
 16.666666666666664]

In [13]:
totals = list(morphpp.sum(1))
totals

[895, 288, 417, 721, 24]

In [ ]:
index = np.arange(len(totals))
barWidth = 0.5
fig,ax = plt.subplots()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_bounds(0,110)
post_patch = ['Nucleated', 'Partial nucleated', 'Outside out', 'No seal', 'Entire cell']
plt.xticks(index, post_patch, rotation = 45, size = 10, ha = 'right')
axes = plt.gca()
axes.set_ylim([0,160])
plt.yticks([0,20,40,60,80,100])

yes = plt.bar(index, y, color = 'blue', width = barWidth)
quest = plt.bar(index, q, color = 'green', bottom = y, width = barWidth)
no = plt.bar(index, n, color = 'red', bottom = [i+j for i,j in zip(y, q)], width = barWidth)
for bar, total in zip(no, totals):
    ax.text(bar.get_x()+bar.get_width()/2, 101, total, ha = 'center', va = 'bottom', fontsize=10)
leg = plt.legend((no,quest,yes), ('No', 'Questionable', 'Yes'), loc = 'upper center', frameon=False)
plt.title('Morphological Calls by Postpatch Outcome')
plt.tight_layout()
plt.show()
fig.savefig('morpho_calls_by_postpatch_outcome', dpi=300)

In [69]:
#yes = y.groupby(['morpho_call', 'post patch class']).count()
#yes = yes.reset_index()
#yes = yes.pivot_table(values = 'Morph', index = 'morpho_call', columns = 'post patch class', aggfunc = 'sum')
#yes